In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hurtos-medelln/hurto_a_persona_medata.csv
/kaggle/input/zonas-tursticas-medelln/atractivos_turisticos.csv


In [2]:
df = pd.read_csv('/kaggle/input/hurtos-medelln/hurto_a_persona_medata.csv', encoding='latin1', delimiter=';')
df.head()

,fecha_hecho,latitud,longitud,caso,valor,cantidad,nombre_barrio,codigo_barrio,codigo_comuna,lugar,...,Unnamed: 23,conducta,modalidad,Unnamed: 26,conducta_especial,arma_medio,bien,categoria_bien,grupo_bien,color
0,1/01/2017 13:00,"6,273576","-75,55377752",VERDADERO,550000,1,Campo Valdés No.2,#0303,3,Vía pública,...,NaN,Hurto a persona,Atraco,NaN,De celular,Objeto contundente,Celular,Tecnología,Mercancía,Sin dato
1,2/01/2017 6:00,"6,22037769","-75,58258977",VERDADERO,2000000,1,Santa Fé,#1504,15,Vehículo particular,...,NaN,Hurto a persona,Rompimiento de ventana,NaN,No,No,Sin dato electrodomésticos,Electrodomésticos,Mercancía,Sin dato
2,11/01/2017 11:00,"6,25043802","-75,56824531",FALSO,60000,1,La Candelaria,#1019,10,Vía pública,...,NaN,Hurto a persona,Atraco,NaN,De celular,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
3,11/01/2017 11:30,"6,25011274","-75,56412008",FALSO,2000000,1,La Candelaria,#1019,10,Vía pública,...,NaN,Hurto a persona,Atraco,NaN,No,Arma cortopunzante,Peso,"Dinero, joyas, piedras preciosas y título valor",Mercancía,Sin dato
4,13/01/2017 4:40,"6,23556805","-75,57362595",VERDADERO,408000,1,Perpetuo Socorro,#1012,10,Vía pública,...,NaN,Hurto a persona,Raponazo,NaN,De celular,No,Celular,Tecnología,Mercancía,Sin dato


In [3]:
import pandas as pd

# Cargar el archivo CSV
# Reemplaza 'ruta/del/archivo.csv' con la ruta de tu archivo
df = pd.read_csv('/kaggle/input/hurtos-medelln/hurto_a_persona_medata.csv', encoding='latin1', delimiter=';')

# Asegurarse de que la columna de fecha esté en formato de fecha (si no, convertirla)
df['fecha_hecho'] = pd.to_datetime(df['fecha_hecho'], errors='coerce')

# 1. Eliminar filas sin fecha (NaT en la columna de fecha)
df = df.dropna(subset=['fecha_hecho'])

# 2. Filtrar y eliminar datos hasta el año 2015
df = df[df['fecha_hecho'].dt.year > 2015]

# 3. Eliminar las columnas no deseadas
columnas_a_eliminar = [
    'grupo_actor', 'actividad_delictiva', 'parentesco', 
    'ocupacion', 'discapacidad', 'grupo_especial', 'estado_civil'
]
df = df.drop(columns=columnas_a_eliminar)

df_personas = df.copy()

# Mostrar las primeras filas para verificar los cambios
print(df_personas.head())

            fecha_hecho     latitud      longitud       caso    valor  \
0   2017-01-01 13:00:00    6,273576  -75,55377752  VERDADERO   550000   
1   2017-02-01 06:00:00  6,22037769  -75,58258977  VERDADERO  2000000   
2   2017-11-01 11:00:00  6,25043802  -75,56824531      FALSO    60000   
3   2017-11-01 11:30:00  6,25011274  -75,56412008      FALSO  2000000   
197 2017-01-02 16:30:00  6,29039298  -75,56468144      FALSO        0   

     cantidad      nombre_barrio codigo_barrio codigo_comuna  \
0           1  Campo Valdés No.2         #0303             3   
1           1           Santa Fé         #1504            15   
2           1      La Candelaria         #1019            10   
3           1      La Candelaria         #1019            10   
197         1      Tricentenario         #0510             5   

                   lugar  ... Unnamed: 23         conducta  \
0            Vía pública  ...         NaN  Hurto a persona   
1    Vehículo particular  ...         NaN  Hurto a p

In [4]:
df_personas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112188 entries, 0 to 351046
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   fecha_hecho        112188 non-null  datetime64[ns]
 1   latitud            91068 non-null   object        
 2   longitud           91068 non-null   object        
 3   caso               112188 non-null  object        
 4   valor              112188 non-null  object        
 5   cantidad           112188 non-null  int64         
 6   nombre_barrio      112188 non-null  object        
 7   codigo_barrio      112188 non-null  object        
 8   codigo_comuna      112188 non-null  object        
 9   lugar              112188 non-null  object        
 10  sede_receptora     112188 non-null  object        
 11  sexo               112188 non-null  object        
 12  edad               112188 non-null  int64         
 13  medio_transporte   112188 non-null  object       

In [5]:
df_personas.describe()

,fecha_hecho,cantidad,edad,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 26
count,112188,112188.000000,112188.000000,0.0,0.0,0.0,0.0
mean,2020-07-15 11:03:44.855599616,1.000250,34.381396,NaN,NaN,NaN,NaN
min,2016-01-01 06:18:00,1.000000,-1.000000,NaN,NaN,NaN,NaN
25%,2018-09-06 06:15:00,1.000000,25.000000,NaN,NaN,NaN,NaN
50%,2020-06-06 01:10:00,1.000000,32.000000,NaN,NaN,NaN,NaN
75%,2022-07-05 12:31:45,1.000000,41.000000,NaN,NaN,NaN,NaN
max,2024-12-09 23:47:00,3.000000,825.000000,NaN,NaN,NaN,NaN
std,NaN,0.016351,13.482276,NaN,NaN,NaN,NaN
